# 🫀 ECG Synthetic Signal Generation using Hybrid CNN-LSTM GAN

## Overview
This notebook implements an advanced **Wasserstein GAN with Gradient Penalty (WGAN-GP)** that combines:
- **1D Convolutional Neural Networks (CNN)** for hierarchical feature extraction
- **Long Short-Term Memory networks (LSTM)** for capturing temporal dependencies
- **Attention mechanisms** (optional) for focusing on important ECG patterns

## Architecture Highlights
- **Generator**: Noise → FC → CNN Upsampling → Bidirectional LSTM → CNN Refinement → ECG Signal
- **Discriminator**: ECG Signal → CNN Features → Bidirectional LSTM → Attention/Classification → Real/Fake
- **Training**: WGAN-GP with gradient penalty for stable training and high-quality signal generation

## Key Features
✅ Temporal coherence through LSTM layers  
✅ Multi-scale feature extraction via CNN  
✅ Stable training with Wasserstein loss  
✅ Comprehensive ECG-specific evaluation metrics  
✅ R-peak detection and heart rate analysis

In [1]:
import pickle
import numpy as np

path_db = "../../ECG_DATASET/dataset_ekg.pkl"

with open(path_db, "rb") as f:
    dataset = pickle.load(f)

dataset

{'NSR': array([[-0.06451476, -0.05951476, -0.02951476, ...,  0.13548524,
          0.13048524,  0.11548524],
        [ 0.24548524,  0.25048524,  0.24048524, ...,  0.34048524,
          0.33548524,  0.31548524],
        [ 0.13048524,  0.12548524,  0.11048524, ..., -0.20451476,
         -0.20451476, -0.20451476],
        ...,
        [-0.46451476, -0.46451476, -0.46451476, ..., -0.59451476,
         -0.58951476, -0.58951476],
        [-0.53451476, -0.51451476, -0.53451476, ..., -0.51451476,
         -0.44451476, -0.40451476],
        [ 0.51548524,  0.50548524,  0.50548524, ...,  0.21048524,
          0.20048524,  0.18548524]]),
 'VT': array([[ 0.10787069,  0.08787069,  0.08787069, ...,  0.01287069,
         -0.00212931,  0.01787069],
        [-0.31212931, -0.29712931, -0.27212931, ...,  0.82787069,
          0.84787069,  0.86787069],
        [ 0.01287069,  0.00787069,  0.00287069, ...,  0.04787069,
          0.06787069,  0.10287069],
        ...,
        [ 0.17287069,  0.16787069,  0.162

In [2]:
X_dataset = dataset['NSR']

In [3]:
N = X_dataset.shape[1]

fs = 360
ts = 1/fs
t = np.arange(N)*ts

In [4]:
# Normalizar al rango [-1, 1] para coincidir con Tanh
X_min = X_dataset.min()
X_max = X_dataset.max()
X_dataset_normalized = 2 * (X_dataset - X_min) / (X_max - X_min) - 1

print(f"Original range: [{X_min:.3f}, {X_max:.3f}]")
print(f"Normalized range: [{X_dataset_normalized.min():.3f}, {X_dataset_normalized.max():.3f}]")

Original range: [-2.270, 3.045]
Normalized range: [-1.000, 1.000]


In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam, AdamW
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 2025
torch.manual_seed(seed)
np.random.seed(seed)

class SineWaveDataset:
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]
    

def get_dataloader(data, batch_size=32, shuffle=True):
    dataset = SineWaveDataset(data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [6]:
# ========== HYBRID CNN-LSTM GAN ARCHITECTURE FOR ECG SIGNALS ==========
# Combining CNNs for feature extraction and LSTMs for temporal dependencies

def weights_init(m):
    """Weight initialization for convolutional and linear layers"""
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find('LSTM') != -1:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)

# ========== GENERATOR WITH CNN + LSTM ==========
class CNNLSTMGenerator(nn.Module):
    """
    Hybrid Generator: CNN for upsampling + LSTM for temporal coherence
    """
    def __init__(self, noise_dim=100, output_length=3600, lstm_hidden=256, lstm_layers=2):
        super().__init__()
        self.noise_dim = noise_dim
        self.output_length = output_length
        self.lstm_hidden = lstm_hidden
        
        # Initial projection: 100 -> 128 * 225
        self.init_size = output_length // 16  # 225 for 3600
        
        self.fc = nn.Sequential(
            nn.Linear(noise_dim, 128 * self.init_size),
            nn.BatchNorm1d(128 * self.init_size),
            nn.ReLU(True)
        )
        
        # CNN upsampling layers: 225 -> 900
        self.conv_upsample = nn.Sequential(
            # 225 -> 450
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            
            # 450 -> 900
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
        )
        
        # LSTM for temporal dependencies (operates on 900 timesteps)
        self.lstm = nn.LSTM(
            input_size=32,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=0.3 if lstm_layers > 1 else 0,
            bidirectional=True  # Bidirectional for better context
        )
        
        # Post-LSTM CNN refinement: 900 -> 3600
        self.post_lstm_conv = nn.Sequential(
            nn.Conv1d(lstm_hidden * 2, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            
            # 900 -> 1800
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            
            # 1800 -> 3600
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
            
            # Final layer
            nn.Conv1d(32, 1, kernel_size=7, padding=3),
            nn.Tanh()
        )
        
    def forward(self, z):
        # Project noise
        x = self.fc(z)
        x = x.view(-1, 128, self.init_size)
        
        # CNN upsampling
        x = self.conv_upsample(x)  # [batch, 32, 900]
        
        # Prepare for LSTM: [batch, seq_len, features]
        x = x.permute(0, 2, 1)  # [batch, 900, 32]
        
        # LSTM processing for temporal coherence
        x, _ = self.lstm(x)  # [batch, 900, lstm_hidden*2]
        
        # Back to CNN format: [batch, features, seq_len]
        x = x.permute(0, 2, 1)  # [batch, lstm_hidden*2, 900]
        
        # Final upsampling and refinement
        x = self.post_lstm_conv(x)  # [batch, 1, 3600]
        
        return x.squeeze(1)

# ========== DISCRIMINATOR WITH CNN + LSTM ==========
class CNNLSTMDiscriminator(nn.Module):
    """
    Hybrid Discriminator: CNN for feature extraction + LSTM for temporal analysis
    """
    def __init__(self, input_length=3600, lstm_hidden=256, lstm_layers=2):
        super().__init__()
        
        # CNN feature extraction: 3600 -> 900
        self.conv_features = nn.Sequential(
            # 3600 -> 1800
            nn.Conv1d(1, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.Dropout(0.2),
            
            # 1800 -> 900
            nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2, True),
            nn.Dropout(0.2),
            
            # 900 -> 900 (refinement)
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, True),
        )
        
        # LSTM for temporal pattern recognition
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=0.3 if lstm_layers > 1 else 0,
            bidirectional=True
        )
        
        # Post-LSTM processing
        self.post_lstm_conv = nn.Sequential(
            nn.Conv1d(lstm_hidden * 2, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, True),
            
            # 900 -> 450
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, True),
        )
        
        # Final classification
        flattened_size = 512 * 450
        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 512),
            nn.LeakyReLU(0.2, True),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )
        
    def forward(self, x):
        # Add channel dimension if needed
        if x.dim() == 2:
            x = x.unsqueeze(1)
        
        # CNN feature extraction
        x = self.conv_features(x)  # [batch, 128, 900]
        
        # Prepare for LSTM
        x = x.permute(0, 2, 1)  # [batch, 900, 128]
        
        # LSTM temporal analysis
        x, _ = self.lstm(x)  # [batch, 900, lstm_hidden*2]
        
        # Back to CNN format
        x = x.permute(0, 2, 1)  # [batch, lstm_hidden*2, 900]
        
        # Post-LSTM convolutions
        x = self.post_lstm_conv(x)  # [batch, 512, 450]
        
        # Flatten and classify
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

# ========== ATTENTION-ENHANCED LSTM DISCRIMINATOR (Alternative) ==========
class AttentionLSTMDiscriminator(nn.Module):
    """
    Discriminator with Self-Attention mechanism for focusing on important ECG features
    """
    def __init__(self, input_length=3600, lstm_hidden=256, lstm_layers=2):
        super().__init__()
        
        # Initial CNN: 3600 -> 900
        self.conv_features = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, True),
        )
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=0.3 if lstm_layers > 1 else 0,
            bidirectional=True
        )
        
        # Self-Attention mechanism
        self.attention_dim = lstm_hidden * 2
        self.attention = nn.Sequential(
            nn.Linear(self.attention_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        # Final classification layers
        self.fc = nn.Sequential(
            nn.Linear(self.attention_dim, 256),
            nn.LeakyReLU(0.2, True),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )
        
    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        
        # CNN feature extraction
        x = self.conv_features(x)  # [batch, 128, 900]
        x = x.permute(0, 2, 1)  # [batch, 900, 128]
        
        # LSTM processing
        lstm_out, _ = self.lstm(x)  # [batch, 900, lstm_hidden*2]
        
        # Self-Attention
        attention_weights = self.attention(lstm_out)  # [batch, 900, 1]
        attention_weights = torch.softmax(attention_weights, dim=1)
        
        # Weighted sum of LSTM outputs
        context = torch.sum(attention_weights * lstm_out, dim=1)  # [batch, lstm_hidden*2]
        
        # Classification
        out = self.fc(context)
        return out

In [7]:
# ========== WGAN-GP TRAINER WITH IMPROVED LOSS ==========
def compute_gradient_penalty(discriminator, real_samples, fake_samples, device):
    """Calculates the gradient penalty for WGAN-GP"""
    batch_size = real_samples.size(0)
    alpha = torch.rand(batch_size, 1).to(device)
    
    # Interpolated samples
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
    
    d_interpolates = discriminator(interpolates)
    
    fake = torch.ones(batch_size, 1).to(device)
    
    # Get gradients
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    
    gradients = gradients.view(batch_size, -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

class WGANGPTrainer:
    """Enhanced Wasserstein GAN with Gradient Penalty trainer"""
    def __init__(self, generator, discriminator, g_optimizer, d_optimizer, device, 
                 lambda_gp=10, n_critic=5):
        self.generator = generator
        self.discriminator = discriminator
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.device = device
        self.lambda_gp = lambda_gp
        self.n_critic = n_critic
        self.d_steps = 0

    def train_step(self, real_data):
        batch_size = real_data.size(0)
        real_data = real_data.to(self.device)

        # ===== Train Discriminator =====
        self.d_optimizer.zero_grad()

        # Real samples
        d_real = self.discriminator(real_data).mean()

        # Fake samples
        noise = torch.randn(batch_size, noise_dim).to(self.device)
        fake_data = self.generator(noise)
        d_fake = self.discriminator(fake_data.detach()).mean()

        # Gradient penalty
        gp = compute_gradient_penalty(self.discriminator, real_data, fake_data, self.device)

        # Wasserstein loss with gradient penalty
        d_loss = d_fake - d_real + self.lambda_gp * gp
        d_loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.discriminator.parameters(), max_norm=1.0)
        self.d_optimizer.step()

        self.d_steps += 1

        # ===== Train Generator (every n_critic steps) =====
        g_loss = torch.tensor(0.0)
        if self.d_steps % self.n_critic == 0:
            self.g_optimizer.zero_grad()
            noise = torch.randn(batch_size, noise_dim).to(self.device)
            fake_data = self.generator(noise)
            g_loss = -self.discriminator(fake_data).mean()
            g_loss.backward()
            
            # Gradient clipping for generator
            torch.nn.utils.clip_grad_norm_(self.generator.parameters(), max_norm=1.0)
            self.g_optimizer.step()

        # Return metrics
        wasserstein_d = (d_real - d_fake).item()
        return d_loss.item(), g_loss.item(), wasserstein_d, gp.item()

# ========== HYPERPARAMETERS ==========
noise_dim = 128  # Increased for more variety
batch_size = 32  # Reduced for LSTM memory efficiency
lstm_hidden = 256
lstm_layers = 2
output_dim = X_dataset_normalized.shape[1]

# ========== MODEL INITIALIZATION ==========
print(f"Device: {device}")
print(f"Training samples: {X_dataset_normalized.shape[0]}")
print(f"Signal length: {output_dim}")
print(f"Batch size: {batch_size}")
print(f"LSTM hidden units: {lstm_hidden}, layers: {lstm_layers}")

# Choose architecture: 'cnn_lstm' or 'attention_lstm'
architecture = 'cnn_lstm'  # Options: 'cnn_lstm', 'attention_lstm'

print("\n" + "="*50)
print(f"Using {architecture.upper()} Architecture")
print("="*50)

# Initialize Generator (always CNN-LSTM hybrid)
generator = CNNLSTMGenerator(
    noise_dim=noise_dim, 
    output_length=output_dim,
    lstm_hidden=lstm_hidden,
    lstm_layers=lstm_layers
).to(device)

# Initialize Discriminator
if architecture == 'attention_lstm':
    discriminator = AttentionLSTMDiscriminator(
        input_length=output_dim,
        lstm_hidden=lstm_hidden,
        lstm_layers=lstm_layers
    ).to(device)
    print("Using Attention-Enhanced LSTM Discriminator")
else:
    discriminator = CNNLSTMDiscriminator(
        input_length=output_dim,
        lstm_hidden=lstm_hidden,
        lstm_layers=lstm_layers
    ).to(device)
    print("Using CNN-LSTM Discriminator")

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nGenerator parameters: {count_parameters(generator):,}")
print(f"Discriminator parameters: {count_parameters(discriminator):,}")

# Initialize weights
generator.apply(weights_init)
discriminator.apply(weights_init)

# ========== OPTIMIZERS WITH SCHEDULER ==========
# Lower learning rates for LSTM stability
g_optimizer = Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
d_optimizer = Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

# Learning rate schedulers
g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    g_optimizer, mode='min', factor=0.5, patience=100, verbose=True
)
d_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    d_optimizer, mode='min', factor=0.5, patience=100, verbose=True
)

# ========== TRAINER SETUP ==========
trainer = WGANGPTrainer(
    generator, discriminator, 
    g_optimizer, d_optimizer, 
    device, 
    lambda_gp=10,
    n_critic=5
)

# ========== TRAINING LOOP ==========
num_epochs = 3000  # More epochs for LSTM convergence
d_losses = []
g_losses = []
w_distances = []
gp_values = []

EPOCHS_TO_PLOT = 100  # Plot more frequently
SAVE_CHECKPOINT_EVERY = 500

# Use normalized data
dataloader_normalized = get_dataloader(X_dataset_normalized, batch_size=batch_size, shuffle=True)

print("\n" + "="*50)
print("Starting CNN-LSTM WGAN-GP Training")
print("="*50 + "\n")

best_w_distance = float('-inf')

for epoch in range(num_epochs):
    epoch_d_loss = []
    epoch_g_loss = []
    epoch_wd = []
    epoch_gp = []
    
    for real_data in dataloader_normalized:
        d_loss, g_loss, w_d, gp = trainer.train_step(real_data)
        epoch_d_loss.append(d_loss)
        epoch_g_loss.append(g_loss if isinstance(g_loss, float) else g_loss.item())
        epoch_wd.append(w_d)
        epoch_gp.append(gp)
    
    # Store average losses per epoch
    avg_d_loss = np.mean(epoch_d_loss)
    avg_g_loss = np.mean(epoch_g_loss)
    avg_wd = np.mean(epoch_wd)
    avg_gp = np.mean(epoch_gp)
    
    d_losses.append(avg_d_loss)
    g_losses.append(avg_g_loss)
    w_distances.append(avg_wd)
    gp_values.append(avg_gp)
    
    # Update learning rate schedulers
    g_scheduler.step(avg_g_loss)
    d_scheduler.step(avg_d_loss)

    # Print progress
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'  D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}')
        print(f'  Wasserstein Distance: {avg_wd:.4f} | GP: {avg_gp:.4f}')
        print(f'  LR - G: {g_optimizer.param_groups[0]["lr"]:.6f} | D: {d_optimizer.param_groups[0]["lr"]:.6f}')
    
    # Save best model
    if avg_wd > best_w_distance:
        best_w_distance = avg_wd
        torch.save({
            'epoch': epoch,
            'generator': generator.state_dict(),
            'discriminator': discriminator.state_dict(),
            'wasserstein_distance': best_w_distance
        }, 'best_ecg_lstm_gan.pth')

    # Plot progress
    if (epoch+1) % EPOCHS_TO_PLOT == 0:
        plt.figure(figsize=(18, 12))
        generator.eval()
        with torch.no_grad():
            # Generate multiple samples
            noise = torch.randn(4, noise_dim).to(device)
            generated_samples = generator(noise).cpu().numpy()
            # Denormalize
            generated_samples_denorm = (generated_samples + 1) / 2 * (X_max - X_min) + X_min
        generator.train()

        # Plot 4 generated samples
        for i in range(4):
            plt.subplot(4, 3, i+1)
            plt.plot(t, generated_samples_denorm[i], color='blue', alpha=0.8, linewidth=0.6)
            plt.xlabel('Time [s]', fontsize=9)
            plt.ylabel('Amplitude', fontsize=9)
            plt.title(f'Generated Sample {i+1} - Epoch {epoch+1}', fontsize=10)
            plt.grid(alpha=0.3)
            plt.xlim([0, 10])
        
        # Plot real samples for comparison
        for i in range(2):
            plt.subplot(4, 3, 5+i)
            idx = np.random.randint(0, X_dataset.shape[0])
            plt.plot(t, X_dataset[idx], color='green', alpha=0.8, linewidth=0.6)
            plt.xlabel('Time [s]', fontsize=9)
            plt.ylabel('Amplitude', fontsize=9)
            plt.title(f'Real ECG Sample {i+1}', fontsize=10)
            plt.grid(alpha=0.3)
            plt.xlim([0, 10])

        # Plot training metrics
        plt.subplot(4, 3, 7)
        plt.plot(d_losses, label='D Loss', alpha=0.7, linewidth=1)
        plt.plot(g_losses, label='G Loss', alpha=0.7, linewidth=1)
        plt.xlabel('Epoch', fontsize=9)
        plt.ylabel('Loss', fontsize=9)
        plt.title('Training Losses', fontsize=10)
        plt.legend(fontsize=8)
        plt.grid(alpha=0.3)

        plt.subplot(4, 3, 8)
        plt.plot(w_distances, color='purple', alpha=0.7, linewidth=1)
        plt.xlabel('Epoch', fontsize=9)
        plt.ylabel('W-Distance', fontsize=9)
        plt.title(f'Wasserstein Distance (Best: {best_w_distance:.3f})', fontsize=10)
        plt.grid(alpha=0.3)

        plt.subplot(4, 3, 9)
        plt.plot(gp_values, color='red', alpha=0.7, linewidth=1)
        plt.xlabel('Epoch', fontsize=9)
        plt.ylabel('Gradient Penalty', fontsize=9)
        plt.title('Gradient Penalty', fontsize=10)
        plt.grid(alpha=0.3)

        # Zoom into ECG segment (2 seconds)
        plt.subplot(4, 3, 10)
        segment_samples = int(2 * fs)  # 2 seconds
        t_segment = t[:segment_samples]
        plt.plot(t_segment, generated_samples_denorm[0][:segment_samples], 
                color='blue', alpha=0.8, linewidth=1, label='Generated')
        plt.plot(t_segment, X_dataset[0][:segment_samples], 
                color='green', alpha=0.6, linewidth=1, label='Real')
        plt.xlabel('Time [s]', fontsize=9)
        plt.ylabel('Amplitude', fontsize=9)
        plt.title('Detailed View (2s)', fontsize=10)
        plt.legend(fontsize=8)
        plt.grid(alpha=0.3)

        # Frequency domain comparison
        plt.subplot(4, 3, 11)
        from scipy.fft import fft, fftfreq
        fft_real = np.abs(fft(X_dataset[0]))[:N//2]
        fft_gen = np.abs(fft(generated_samples_denorm[0]))[:N//2]
        freqs = fftfreq(N, ts)[:N//2]
        plt.semilogy(freqs, fft_real, label='Real', alpha=0.7, linewidth=1)
        plt.semilogy(freqs, fft_gen, label='Generated', alpha=0.7, linewidth=1)
        plt.xlabel('Frequency [Hz]', fontsize=9)
        plt.ylabel('Magnitude (log)', fontsize=9)
        plt.title('Frequency Spectrum', fontsize=10)
        plt.xlim([0, 50])
        plt.legend(fontsize=8)
        plt.grid(alpha=0.3)

        # Statistical comparison
        plt.subplot(4, 3, 12)
        real_mean = X_dataset.mean(axis=1)
        gen_mean = generated_samples_denorm.mean(axis=1)
        plt.hist(real_mean, bins=30, alpha=0.5, label='Real Mean', density=True)
        plt.hist(gen_mean, bins=30, alpha=0.5, label='Gen Mean', density=True)
        plt.xlabel('Mean Amplitude', fontsize=9)
        plt.ylabel('Density', fontsize=9)
        plt.title('Distribution Comparison', fontsize=10)
        plt.legend(fontsize=8)
        plt.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.show()

print("\n" + "="*50)
print("Training Finished!")
print(f"Best Wasserstein Distance: {best_w_distance:.4f}")
print("="*50)

Device: cuda
Training samples: 283
Signal length: 3600
Batch size: 32
LSTM hidden units: 256, layers: 2

Using CNN_LSTM Architecture
Using CNN-LSTM Discriminator

Generator parameters: 6,223,393
Discriminator parameters: 121,286,625
Using CNN-LSTM Discriminator

Generator parameters: 6,223,393
Discriminator parameters: 121,286,625


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

In [ ]:
# ========== COMPREHENSIVE EVALUATION METRICS FOR ECG SIGNALS ==========
from scipy.stats import pearsonr, wasserstein_distance as wd, ks_2samp
from scipy.signal import correlate, find_peaks
from scipy.fft import fft, fftfreq

def calculate_ecg_metrics(real_data, generated_data, fs=360):
    """
    Calculate comprehensive metrics to evaluate ECG quality
    """
    metrics = {}
    
    # 1. Statistical Metrics
    metrics['MSE'] = np.mean((real_data - generated_data) ** 2)
    metrics['MAE'] = np.mean(np.abs(real_data - generated_data))
    metrics['RMSE'] = np.sqrt(metrics['MSE'])
    
    # 2. Correlation Metrics
    corr, p_val = pearsonr(real_data.flatten(), generated_data.flatten())
    metrics['Pearson_Correlation'] = corr
    metrics['Pearson_P_Value'] = p_val
    
    # Cross-correlation
    cross_corr = correlate(real_data.flatten(), generated_data.flatten(), mode='valid')
    metrics['Max_Cross_Correlation'] = np.max(cross_corr) / (np.linalg.norm(real_data) * np.linalg.norm(generated_data))
    
    # 3. Distribution Similarity
    ks_stat, ks_p = ks_2samp(real_data.flatten(), generated_data.flatten())
    metrics['KS_Statistic'] = ks_stat
    metrics['KS_P_Value'] = ks_p
    
    # Wasserstein distance
    metrics['Wasserstein_Distance'] = wd(real_data.flatten(), generated_data.flatten())
    
    # 4. Frequency Domain Analysis
    fft_real = np.abs(fft(real_data.flatten()))
    fft_gen = np.abs(fft(generated_data.flatten()))
    
    # Frequency similarity (correlation of spectra)
    freq_corr, _ = pearsonr(fft_real, fft_gen)
    metrics['Frequency_Correlation'] = freq_corr
    
    # Power spectral density comparison
    psd_real = fft_real ** 2
    psd_gen = fft_gen ** 2
    metrics['PSD_MSE'] = np.mean((psd_real - psd_gen) ** 2)
    
    # 5. Temporal Coherence
    # Measure autocorrelation similarity
    autocorr_real = correlate(real_data.flatten(), real_data.flatten(), mode='full')
    autocorr_gen = correlate(generated_data.flatten(), generated_data.flatten(), mode='full')
    autocorr_real = autocorr_real[len(autocorr_real)//2:][:1000]
    autocorr_gen = autocorr_gen[len(autocorr_gen)//2:][:1000]
    metrics['Autocorrelation_Similarity'] = pearsonr(autocorr_real, autocorr_gen)[0]
    
    # 6. ECG-Specific Metrics (R-peak detection)
    try:
        # Detect R-peaks in real data
        real_peaks, _ = find_peaks(real_data.flatten(), distance=fs*0.6, prominence=0.3)
        gen_peaks, _ = find_peaks(generated_data.flatten(), distance=fs*0.6, prominence=0.3)
        
        # Heart rate estimation
        if len(real_peaks) > 1:
            real_hr = 60 / (np.mean(np.diff(real_peaks)) / fs)
            metrics['Real_Heart_Rate'] = real_hr
        
        if len(gen_peaks) > 1:
            gen_hr = 60 / (np.mean(np.diff(gen_peaks)) / fs)
            metrics['Generated_Heart_Rate'] = gen_hr
            metrics['Heart_Rate_Difference'] = abs(real_hr - gen_hr) if len(real_peaks) > 1 else np.nan
        
        metrics['Real_R_Peaks'] = len(real_peaks)
        metrics['Generated_R_Peaks'] = len(gen_peaks)
        metrics['R_Peak_Count_Difference'] = abs(len(real_peaks) - len(gen_peaks))
    except:
        metrics['R_Peak_Detection'] = 'Failed'
    
    return metrics

def plot_detailed_comparison(real_samples, gen_samples, t, fs=360):
    """Create detailed visualization comparing real and generated ECG signals"""
    plt.figure(figsize=(18, 12))
    
    # Plot 1: Overlaid signals
    plt.subplot(3, 3, 1)
    for i in range(min(3, len(gen_samples))):
        plt.plot(t, gen_samples[i], alpha=0.5, linewidth=0.5, label=f'Gen {i+1}')
    for i in range(min(3, len(real_samples))):
        plt.plot(t, real_samples[i], alpha=0.5, linewidth=0.5, linestyle='--', label=f'Real {i+1}')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.title('Signal Overlay')
    plt.legend(fontsize=7)
    plt.grid(alpha=0.3)
    plt.xlim([0, 3])
    
    # Plot 2: Average signals
    plt.subplot(3, 3, 2)
    plt.plot(t, real_samples.mean(axis=0), color='green', linewidth=1.5, label='Real Avg', alpha=0.8)
    plt.fill_between(t, 
                     real_samples.mean(axis=0) - real_samples.std(axis=0),
                     real_samples.mean(axis=0) + real_samples.std(axis=0),
                     color='green', alpha=0.2)
    plt.plot(t, gen_samples.mean(axis=0), color='blue', linewidth=1.5, label='Gen Avg', alpha=0.8)
    plt.fill_between(t, 
                     gen_samples.mean(axis=0) - gen_samples.std(axis=0),
                     gen_samples.mean(axis=0) + gen_samples.std(axis=0),
                     color='blue', alpha=0.2)
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.title('Mean ± Std')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.xlim([0, 3])
    
    # Plot 3: Power Spectral Density
    plt.subplot(3, 3, 3)
    freqs = fftfreq(len(t), 1/fs)[:len(t)//2]
    psd_real = np.mean([np.abs(fft(sig))**2 for sig in real_samples], axis=0)[:len(t)//2]
    psd_gen = np.mean([np.abs(fft(sig))**2 for sig in gen_samples], axis=0)[:len(t)//2]
    plt.semilogy(freqs, psd_real, label='Real', alpha=0.7, linewidth=1.5)
    plt.semilogy(freqs, psd_gen, label='Generated', alpha=0.7, linewidth=1.5)
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Power Spectral Density')
    plt.title('PSD Comparison')
    plt.xlim([0, 50])
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Plot 4-6: Distribution comparisons
    metrics_to_plot = ['mean', 'std', 'max']
    for idx, metric in enumerate(metrics_to_plot):
        plt.subplot(3, 3, 4+idx)
        if metric == 'mean':
            real_vals = real_samples.mean(axis=1)
            gen_vals = gen_samples.mean(axis=1)
        elif metric == 'std':
            real_vals = real_samples.std(axis=1)
            gen_vals = gen_samples.std(axis=1)
        else:
            real_vals = real_samples.max(axis=1)
            gen_vals = gen_samples.max(axis=1)
        
        plt.hist(real_vals, bins=20, alpha=0.5, label='Real', density=True, color='green')
        plt.hist(gen_vals, bins=20, alpha=0.5, label='Generated', density=True, color='blue')
        plt.xlabel(f'{metric.capitalize()} Value')
        plt.ylabel('Density')
        plt.title(f'{metric.capitalize()} Distribution')
        plt.legend()
        plt.grid(alpha=0.3)
    
    # Plot 7: Autocorrelation
    plt.subplot(3, 3, 7)
    lag_max = int(2 * fs)  # 2 seconds
    autocorr_real = correlate(real_samples[0], real_samples[0], mode='full')
    autocorr_real = autocorr_real[len(autocorr_real)//2:][:lag_max]
    autocorr_gen = correlate(gen_samples[0], gen_samples[0], mode='full')
    autocorr_gen = autocorr_gen[len(autocorr_gen)//2:][:lag_max]
    lags = np.arange(lag_max) / fs
    plt.plot(lags, autocorr_real / autocorr_real[0], label='Real', linewidth=1.5, alpha=0.7)
    plt.plot(lags, autocorr_gen / autocorr_gen[0], label='Generated', linewidth=1.5, alpha=0.7)
    plt.xlabel('Lag [s]')
    plt.ylabel('Normalized Autocorrelation')
    plt.title('Autocorrelation Function')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Plot 8: R-peak detection visualization
    plt.subplot(3, 3, 8)
    signal_segment = real_samples[0][:int(3*fs)]
    t_segment = t[:int(3*fs)]
    peaks_real, _ = find_peaks(signal_segment, distance=fs*0.6, prominence=0.3)
    plt.plot(t_segment, signal_segment, color='green', linewidth=1, alpha=0.7, label='Real')
    plt.plot(t_segment[peaks_real], signal_segment[peaks_real], 'ro', markersize=8, label='R-peaks')
    
    signal_segment_gen = gen_samples[0][:int(3*fs)]
    peaks_gen, _ = find_peaks(signal_segment_gen, distance=fs*0.6, prominence=0.3)
    plt.plot(t_segment, signal_segment_gen, color='blue', linewidth=1, alpha=0.7, label='Generated')
    plt.plot(t_segment[peaks_gen], signal_segment_gen[peaks_gen], 'bs', markersize=8, label='Detected peaks')
    
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.title('R-Peak Detection')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Plot 9: Q-Q plot
    plt.subplot(3, 3, 9)
    from scipy import stats
    real_sorted = np.sort(real_samples.flatten())
    gen_sorted = np.sort(gen_samples.flatten())
    # Subsample for clarity
    n_points = min(1000, len(real_sorted))
    indices = np.linspace(0, len(real_sorted)-1, n_points, dtype=int)
    plt.scatter(real_sorted[indices], gen_sorted[indices], alpha=0.3, s=1)
    plt.plot([real_sorted.min(), real_sorted.max()], 
             [real_sorted.min(), real_sorted.max()], 
             'r--', linewidth=2, label='Perfect match')
    plt.xlabel('Real Quantiles')
    plt.ylabel('Generated Quantiles')
    plt.title('Q-Q Plot')
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# ========== EVALUATION ==========
print("\n" + "="*50)
print("COMPREHENSIVE ECG EVALUATION")
print("="*50 + "\n")

generator.eval()
num_samples = 200  # More samples for better statistics

with torch.no_grad():
    noise = torch.randn(num_samples, noise_dim).to(device)
    generated_samples = generator(noise).cpu().numpy()
    # Denormalize
    generated_samples = (generated_samples + 1) / 2 * (X_max - X_min) + X_min

# Select real samples
real_samples = X_dataset[:num_samples]

# Calculate comprehensive metrics
print("Calculating metrics...")
metrics = calculate_ecg_metrics(real_samples, generated_samples, fs=fs)

print("\n📊 EVALUATION RESULTS:")
print("-" * 50)
print("\n🔢 Statistical Metrics:")
for key in ['MSE', 'MAE', 'RMSE']:
    if key in metrics:
        print(f"  {key}: {metrics[key]:.6f}")

print("\n📈 Correlation Metrics:")
for key in ['Pearson_Correlation', 'Max_Cross_Correlation', 'Frequency_Correlation', 'Autocorrelation_Similarity']:
    if key in metrics:
        print(f"  {key}: {metrics[key]:.6f}")

print("\n📊 Distribution Metrics:")
for key in ['KS_Statistic', 'Wasserstein_Distance']:
    if key in metrics:
        print(f"  {key}: {metrics[key]:.6f}")

print("\n💓 ECG-Specific Metrics:")
for key in ['Real_Heart_Rate', 'Generated_Heart_Rate', 'Heart_Rate_Difference', 
            'Real_R_Peaks', 'Generated_R_Peaks', 'R_Peak_Count_Difference']:
    if key in metrics:
        print(f"  {key}: {metrics[key]:.2f}")

print("\n📉 Statistical Comparison:")
print(f"  Real data     - Mean: {real_samples.mean():.4f}, Std: {real_samples.std():.4f}")
print(f"  Generated     - Mean: {generated_samples.mean():.4f}, Std: {generated_samples.std():.4f}")
print(f"  Real data     - Min: {real_samples.min():.4f}, Max: {real_samples.max():.4f}")
print(f"  Generated     - Min: {generated_samples.min():.4f}, Max: {generated_samples.max():.4f}")

# Create detailed visualization
plot_detailed_comparison(real_samples[:50], generated_samples[:50], t, fs=fs)

# Save final model and results
print("\n" + "="*50)
print("Saving models and results...")
torch.save({
    'generator': generator.state_dict(),
    'discriminator': discriminator.state_dict(),
    'g_optimizer': g_optimizer.state_dict(),
    'd_optimizer': d_optimizer.state_dict(),
    'epoch': num_epochs,
    'metrics': metrics,
    'architecture': architecture,
    'noise_dim': noise_dim,
    'lstm_hidden': lstm_hidden,
    'lstm_layers': lstm_layers
}, 'ecg_cnn_lstm_wgan_gp_final.pth')
print("✅ Model saved to: ecg_cnn_lstm_wgan_gp_final.pth")

# Save metrics to CSV
import pandas as pd
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('ecg_lstm_gan_metrics.csv', index=False)
print("✅ Metrics saved to: ecg_lstm_gan_metrics.csv")
print("="*50)

In [ ]:
# ========== GENERATE SYNTHETIC ECG DATASET ==========
print("\n" + "="*50)
print("GENERATING SYNTHETIC ECG DATASET")
print("="*50 + "\n")

# Number of synthetic signals to generate
num_synthetic = 500

print(f"Generating {num_synthetic} synthetic ECG signals...")
generator.eval()
synthetic_signals = []

batch_size_gen = 50
with torch.no_grad():
    for i in range(0, num_synthetic, batch_size_gen):
        batch_size_current = min(batch_size_gen, num_synthetic - i)
        noise = torch.randn(batch_size_current, noise_dim).to(device)
        generated = generator(noise).cpu().numpy()
        # Denormalize
        generated = (generated + 1) / 2 * (X_max - X_min) + X_min
        synthetic_signals.append(generated)
        
        if (i + batch_size_current) % 100 == 0:
            print(f"  Generated {i + batch_size_current}/{num_synthetic} signals...")

synthetic_signals = np.vstack(synthetic_signals)
print(f"✅ Successfully generated {synthetic_signals.shape[0]} signals")
print(f"   Shape: {synthetic_signals.shape}")

# Visualize a grid of synthetic signals
print("\nVisualizing synthetic signals...")
fig, axes = plt.subplots(5, 4, figsize=(20, 15))
axes = axes.flatten()

for i in range(20):
    ax = axes[i]
    ax.plot(t[:int(2*fs)], synthetic_signals[i][:int(2*fs)], color='blue', linewidth=0.8, alpha=0.8)
    ax.set_xlabel('Time [s]', fontsize=8)
    ax.set_ylabel('Amplitude', fontsize=8)
    ax.set_title(f'Synthetic ECG #{i+1}', fontsize=9)
    ax.grid(alpha=0.3)
    ax.set_xlim([0, 2])

plt.tight_layout()
plt.suptitle('Generated Synthetic ECG Signals (2-second segments)', fontsize=14, y=1.001)
plt.show()

# Save synthetic dataset
print("\n" + "="*50)
print("Saving synthetic dataset...")

# Save as numpy array
np.save('synthetic_ecg_signals.npy', synthetic_signals)
print("✅ Saved as: synthetic_ecg_signals.npy")

# Save as pickle (same format as original dataset)
synthetic_dataset = {
    'NSR_synthetic': synthetic_signals,
    'metadata': {
        'num_signals': synthetic_signals.shape[0],
        'signal_length': synthetic_signals.shape[1],
        'sampling_rate': fs,
        'duration_seconds': synthetic_signals.shape[1] / fs,
        'generation_method': 'CNN-LSTM WGAN-GP',
        'architecture': architecture,
        'noise_dim': noise_dim,
        'lstm_hidden': lstm_hidden,
        'lstm_layers': lstm_layers
    }
}

with open('synthetic_ecg_dataset.pkl', 'wb') as f:
    pickle.dump(synthetic_dataset, f)
print("✅ Saved as: synthetic_ecg_dataset.pkl")

# Quality assessment of synthetic dataset
print("\n" + "="*50)
print("SYNTHETIC DATASET QUALITY ASSESSMENT")
print("="*50)

# Compare distributions
print("\n📊 Statistical Properties:")
print(f"  Real dataset:")
print(f"    Mean: {X_dataset.mean():.4f} ± {X_dataset.std():.4f}")
print(f"    Range: [{X_dataset.min():.4f}, {X_dataset.max():.4f}]")
print(f"  Synthetic dataset:")
print(f"    Mean: {synthetic_signals.mean():.4f} ± {synthetic_signals.std():.4f}")
print(f"    Range: [{synthetic_signals.min():.4f}, {synthetic_signals.max():.4f}]")

# Heart rate analysis for synthetic signals
print("\n💓 Heart Rate Analysis (sample of 50 signals):")
heart_rates_synthetic = []
for i in range(min(50, len(synthetic_signals))):
    try:
        peaks, _ = find_peaks(synthetic_signals[i], distance=fs*0.6, prominence=0.3)
        if len(peaks) > 1:
            hr = 60 / (np.mean(np.diff(peaks)) / fs)
            if 40 < hr < 120:  # Physiologically reasonable range
                heart_rates_synthetic.append(hr)
    except:
        pass

if heart_rates_synthetic:
    print(f"  Average HR: {np.mean(heart_rates_synthetic):.1f} ± {np.std(heart_rates_synthetic):.1f} bpm")
    print(f"  Range: [{np.min(heart_rates_synthetic):.1f}, {np.max(heart_rates_synthetic):.1f}] bpm")
    print(f"  Valid signals: {len(heart_rates_synthetic)}/50")

# Final comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Distribution of means
axes[0, 0].hist(X_dataset.mean(axis=1), bins=40, alpha=0.5, label='Real', density=True, color='green')
axes[0, 0].hist(synthetic_signals.mean(axis=1), bins=40, alpha=0.5, label='Synthetic', density=True, color='blue')
axes[0, 0].set_xlabel('Mean Amplitude')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Distribution of Signal Means')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Plot 2: Distribution of stds
axes[0, 1].hist(X_dataset.std(axis=1), bins=40, alpha=0.5, label='Real', density=True, color='green')
axes[0, 1].hist(synthetic_signals.std(axis=1), bins=40, alpha=0.5, label='Synthetic', density=True, color='blue')
axes[0, 1].set_xlabel('Standard Deviation')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Distribution of Signal Std Dev')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Plot 3: Sample real vs synthetic
axes[1, 0].plot(t[:int(3*fs)], X_dataset[0][:int(3*fs)], color='green', linewidth=1.2, alpha=0.8, label='Real')
axes[1, 0].plot(t[:int(3*fs)], synthetic_signals[0][:int(3*fs)], color='blue', linewidth=1.2, alpha=0.8, label='Synthetic')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('Sample Comparison (3 seconds)')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Plot 4: Average PSD
axes[1, 1].set_title('Average Power Spectral Density')
freqs = fftfreq(N, ts)[:N//2]
psd_real_avg = np.mean([np.abs(fft(sig))**2 for sig in X_dataset[:50]], axis=0)[:N//2]
psd_syn_avg = np.mean([np.abs(fft(sig))**2 for sig in synthetic_signals[:50]], axis=0)[:N//2]
axes[1, 1].semilogy(freqs, psd_real_avg, label='Real', linewidth=1.5, alpha=0.8, color='green')
axes[1, 1].semilogy(freqs, psd_syn_avg, label='Synthetic', linewidth=1.5, alpha=0.8, color='blue')
axes[1, 1].set_xlabel('Frequency [Hz]')
axes[1, 1].set_ylabel('Power')
axes[1, 1].set_xlim([0, 50])
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("✅ PROCESS COMPLETED SUCCESSFULLY!")
print("="*50)
print("\n📁 Generated Files:")
print("  1. ecg_cnn_lstm_wgan_gp_final.pth - Trained model")
print("  2. best_ecg_lstm_gan.pth - Best model checkpoint")
print("  3. synthetic_ecg_signals.npy - Synthetic signals (numpy)")
print("  4. synthetic_ecg_dataset.pkl - Synthetic dataset (pickle)")
print("  5. ecg_lstm_gan_metrics.csv - Evaluation metrics")
print("\n🎉 Ready for downstream tasks (classification, augmentation, etc.)")
print("="*50)